# Get Started

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Kqlmagic:__ 0.1.90<br>
__Platforms Supported:__<br>
    -  Azure Notebooks Free Compute
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  Log Analytics - SecurityEvent
    
### Description
The notebook provides you basic knowledge to use Azure Sentinel Notebooks.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Table of Contents

1. Retrieve Log Analytics information 
2. Install Kqlmagic
3. Log into Log Analytics
4. Get a list of available data tables from Log Analytics
5. Run a simple Kusto query

## 1. Retrieve Log Analytics Information

In [ ]:
path = %env PATH
dsvm = False
if '/dsvm/' in path:
    dsvm = True
    
if dsvm == False:
    # Run this if you are using Free Compute
    from utils import config_reader as mod
    workspace_id = mod.config_reader.read_config_values("config.json")[3]
    tenant_id = mod.config_reader.read_config_values("config.json")[0]

    ######################################################################
    your_workspace_name = mod.config_reader.read_config_values("config.json")[4]
    print("Your Log Analytic Workspace: " + your_workspace_name)
    ######################################################################
else:
    # Run this if you are using DSVM.  You need to copy the values from config.json, if the file has no value, then you need to go to Log Analytics Portal to get the information.
    tenant_id = input('tenant_id:')
    subscription_id = input('subscription_id:')
    resource_group = input('resource_group:')
    workspace_id = input('workspace_id:')
    workspace_name = input('workspace_name:')

## 2. Install Required Packages
These are the two core packages required by most Azure Sentinel Notebooks. You need to install these only once in each Azure Notebooks project (or other Python environment) but they are both updated regularly so it is worth running<br>
```pip install {pkg} --upgrade```<br>
every few weeks to get the latest features and fixes.

In [ ]:
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)
    
# You may needs these - should only need to uncomment and run once

!pip install Kqlmagic --no-cache-dir --upgrade --user

### 2.1 Install KqlMagic 
For more information and sample notebooks go to [Kqlmagic github](https://github.com/Microsoft/jupyter-Kqlmagic)

In [ ]:
# run only once unless upgrade
!pip install Kqlmagic --upgrade

In [ ]:
# Make sure you have 0.1.90 or above, if not, run Kqlmagic installation again
%reload_ext Kqlmagic

If you need any help for kql magic     --> %kql --help "help" -popup_window
If you need any help for Log Analytics --> %kql --help "LogAnalytics"

### 2.2 Install Sentinel Security Tools
This is open source Python tools for working in Notebooks. It includes modules and classes for querying and analyzing data in the context of a security investigation or security hunting. 

For more information go the [msticpy github](https://github.com/microsoft/msticpy)

In [ ]:
# Provisional location prior to migration to PyPi
!pip install git+https://github.com/microsoft/msticpy --upgrade --user

## 3. Log into Log Analytics

**A Note about authenticating to Log Analytics**<br>
The most typical authentication method is user/device authentication, where you authenticate using your Azure Active Directory credentials and provide and one-time device ID that is bound to the authentication.<br>
If you are using user/device authentication, run the following cell. 
- Click the 'Copy code to clipboard and authenticate' button.
- This will pop up an Azure Active Directory authentication dialog (in a new tab or browser window). The device code will have been copied to the clipboard. 
- Select the text box and paste (Ctrl-V/Cmd-V) the copied value. 
- You should then be redirected to a user authentication page where you should authenticate with a user account that has permission to query your Log Analytics workspace.

Use the following syntax if you are authenticating using an Azure Active Directory AppId and Secret:
```
%kql loganalytics://tenant(aad_tenant).workspace(WORKSPACE_ID).clientid(client_id).clientsecret(client_secret)
```
instead of
```
%kql loganalytics://code().workspace(WORKSPACE_ID)```
See the KqlMagic github (above for more details)

In [ ]:
%kql loganalytics://code;workspace=workspace_id;tenant=tenant_id;alias="asidb"

## 4. Get a list of available data tables from Log Analytics

1. Tables with records

In [ ]:
%kql search * | summarize count() by Type

2. All Tables in drop down

In [ ]:
import ipywidgets as widgets
from IPython.display import display
dbSchema = %kql --schema "asidb@loganalytics"
tables = list(dbSchema.keys())
selectedTable = widgets.Dropdown(options=tables, value=tables[24],description='Data Table:')
display(selectedTable)

## 5. Run a simple Kusto query

In [ ]:
%kql SecurityEvent | take 5 | project Account, Computer
df = _kql_raw_result_.to_dataframe()
df